<a href="https://colab.research.google.com/github/wtsyang/dl-reproducibility-project/blob/pipeline/cnnTrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from google.colab import files

uploaded = files.upload()

Saving Model.py to Model.py


In [0]:
from Model import Model

In [0]:
try:
    from torchsummary import summary
except ModuleNotFoundError:
    !pip install -q torchsummary
    from torchsummary import summary


In [0]:
try:
    import torch
except ModuleNotFoundError:
    from os import path
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

    accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.1-{platform}-linux_x86_64.whl
    import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

In [0]:
try:
    import torchvision
except ModuleNotFoundError:
    !pip install -q torchvision

from torchvision import datasets, transforms

In [0]:
cuda = True
train_batch_size = 32
test_batch_size = 124
best_loss = float("inf")
best_epoch = -1
dataset_path = './cifar10'
gsync_save = True

In [0]:
if gsync_save:
    try:
        import utils
    except ModuleNotFoundError:
        !wget https://raw.githubusercontent.com/StefOe/colab-pytorch-utils/HEAD/utils.py
        import utils

    gsync = utils.GDriveSync()

--2020-03-06 20:06:37--  https://raw.githubusercontent.com/StefOe/colab-pytorch-utils/HEAD/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4519 (4.4K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]   4.41K  --.-KB/s    in 0s      

2020-03-06 20:06:37 (140 MB/s) - ‘utils.py’ saved [4519/4519]



In [0]:
cuda = cuda and torch.cuda.is_available()
trainset = datasets.CIFAR10(root=dataset_path, train=True, download=True)
train_mean = trainset.data.mean(axis=(0,1,2))/255  # [0.49139968  0.48215841  0.44653091]
train_std = trainset.data.std(axis=(0,1,2))/255  # [0.24703223  0.24348513  0.26158784]
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(train_mean, train_std),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(train_mean, train_std),
])
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
train_loader = torch.utils.data.DataLoader(datasets.CIFAR10(
    root=dataset_path, train=True, download=True,
    transform=transform_train),
    batch_size=train_batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(root=dataset_path, train=False, download=True,
    transform=transform_test),
    batch_size=test_batch_size, shuffle=False, **kwargs)


Extracting ./cifar10/cifar-10-python.tar.gz to ./cifar10
Files already downloaded and verified
Files already downloaded and verified


In [0]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data))

In [0]:
def test(epoch, best_loss, best_epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)

        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).data
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(
        '\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset), 100. * correct /
            len(test_loader.dataset)))
    
    if test_loss < best_loss:
        best_epoch = epoch
        best_loss = test_loss
        torch.save(model, "best.pt")
        if gsync_save:
            gsync.update_file_to_folder("best.pt")
    return best_loss, best_epoch

In [0]:
try: 
  del model
except:
  print('')
# Choose Model A by basemodel=[True, False, False] and choose B by basemodel=[False, True, False]
# Choose the modification by modifiedModel=[True, False, False, False]. The first one is the model without any modification.
# The scecond one is the strided. The third: ConvPool. The last: All CNN.
model = Model()
if cuda:
    model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
scheduler = optim.lr_scheduler.MultiStepLR(
    optimizer, milestones=[200, 250, 300], gamma=0.1)

In [0]:
print(model)

Model(
  (model): Sequential(
    (0): Dropout(p=0.2, inplace=False)
    (1): Conv2d(3, 96, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (2): ReLU()
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.5, inplace=False)
    (5): Conv2d(96, 192, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (6): ReLU()
    (7): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Dropout(p=0.5, inplace=False)
    (9): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU()
    (11): Conv2d(192, 192, kernel_size=(1, 1), stride=(1, 1))
    (12): ReLU()
    (13): Conv2d(192, 10, kernel_size=(1, 1), stride=(1, 1))
    (14): ReLU()
    (15): AdaptiveAvgPool2d(output_size=1)
    (16): Flatten()
  )
  (conv_3_192_192_1): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_1_192_192): Conv2d(192, 192, kernel_size=(1, 1), stride=(1, 1))
  (conv_1_192_class): C

In [0]:
summary(model.model, input_size=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Dropout-1            [-1, 3, 32, 32]               0
            Conv2d-2           [-1, 96, 30, 30]           7,296
              ReLU-3           [-1, 96, 30, 30]               0
         MaxPool2d-4           [-1, 96, 14, 14]               0
           Dropout-5           [-1, 96, 14, 14]               0
            Conv2d-6          [-1, 192, 12, 12]         460,992
              ReLU-7          [-1, 192, 12, 12]               0
         MaxPool2d-8            [-1, 192, 5, 5]               0
           Dropout-9            [-1, 192, 5, 5]               0
           Conv2d-10            [-1, 192, 5, 5]         331,968
             ReLU-11            [-1, 192, 5, 5]               0
           Conv2d-12            [-1, 192, 5, 5]          37,056
             ReLU-13            [-1, 192, 5, 5]               0
           Conv2d-14             [-1, 1

In [0]:
for epoch in range(350):
    scheduler.step()
    train(epoch)
    best_loss, best_epoch = test(epoch, best_loss, best_epoch)

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Train Epoch: 0 [0/50000 (0%)]	Loss: 2.303414
Train Epoch: 0 [25600/50000 (51%)]	Loss: 2.075816

Test set: Average loss: 0.0149, Accuracy: 3249/10000 (32%)



Uploading file best.pt: 100%|██████████| 100/100 [00:01<00:00, 82.34it/s]


Train Epoch: 1 [0/50000 (0%)]	Loss: 1.910877
Train Epoch: 1 [25600/50000 (51%)]	Loss: 1.695527

Test set: Average loss: 0.0130, Accuracy: 4298/10000 (43%)



Uploading file best.pt: 100%|██████████| 100/100 [00:01<00:00, 61.67it/s]


...

Train Epoch: 233 [0/50000 (0%)]	Loss: 0.583615
Train Epoch: 233 [25600/50000 (51%)]	Loss: 0.588380

Test set: Average loss: 0.0054, Accuracy: 7851/10000 (79%)

Train Epoch: 234 [0/50000 (0%)]	Loss: 0.603386
Train Epoch: 234 [25600/50000 (51%)]	Loss: 0.619180

Test set: Average loss: 0.0055, Accuracy: 7824/10000 (78%)

Train Epoch: 235 [0/50000 (0%)]	Loss: 0.603425
Train Epoch: 235 [25600/50000 (51%)]	Loss: 0.599515

Test set: Average loss: 0.0054, Accuracy: 7855/10000 (79%)

Train Epoch: 236 [0/50000 (0%)]	Loss: 0.570486
Train Epoch: 236 [25600/50000 (51%)]	Loss: 0.618891

Test set: Average loss: 0.0054, Accuracy: 7876/10000 (79%)

Train Epoch: 237 [0/50000 (0%)]	Loss: 0.585611
Train Epoch: 237 [25600/50000 (51%)]	Loss: 0.540469

Test set: Average loss: 0.0054, Accuracy: 7889/10000 (79%)

Train Epoch: 238 [0/50000 (0%)]	Loss: 0.509158
Train Epoch: 238 [25600/50000 (51%)]	Loss: 0.751793

Test set: Average loss: 0.0053, Accuracy: 7876/10000 (79%)

Train Epoch: 239 [0/50000 (0%)]	Loss: 0.

Uploading file best.pt: 100%|██████████| 100/100 [00:01<00:00, 91.13it/s]


Train Epoch: 258 [0/50000 (0%)]	Loss: 0.567819
Train Epoch: 258 [25600/50000 (51%)]	Loss: 0.632363

Test set: Average loss: 0.0053, Accuracy: 7913/10000 (79%)

Train Epoch: 259 [0/50000 (0%)]	Loss: 0.559609
Train Epoch: 259 [25600/50000 (51%)]	Loss: 0.708812

Test set: Average loss: 0.0053, Accuracy: 7925/10000 (79%)

Train Epoch: 260 [0/50000 (0%)]	Loss: 0.593296
Train Epoch: 260 [25600/50000 (51%)]	Loss: 0.558032

Test set: Average loss: 0.0053, Accuracy: 7927/10000 (79%)

Train Epoch: 261 [0/50000 (0%)]	Loss: 0.708984
Train Epoch: 261 [25600/50000 (51%)]	Loss: 0.577536

Test set: Average loss: 0.0053, Accuracy: 7924/10000 (79%)

Train Epoch: 262 [0/50000 (0%)]	Loss: 0.555146
Train Epoch: 262 [25600/50000 (51%)]	Loss: 0.636425

Test set: Average loss: 0.0053, Accuracy: 7925/10000 (79%)

Train Epoch: 263 [0/50000 (0%)]	Loss: 0.541696
Train Epoch: 263 [25600/50000 (51%)]	Loss: 0.648384

Test set: Average loss: 0.0053, Accuracy: 7923/10000 (79%)

Train Epoch: 264 [0/50000 (0%)]	Loss: 0.

Uploading file best.pt: 100%|██████████| 100/100 [00:01<00:00, 83.79it/s]


Train Epoch: 278 [0/50000 (0%)]	Loss: 0.555720
Train Epoch: 278 [25600/50000 (51%)]	Loss: 0.651546

Test set: Average loss: 0.0053, Accuracy: 7911/10000 (79%)

Train Epoch: 279 [0/50000 (0%)]	Loss: 0.579255
Train Epoch: 279 [25600/50000 (51%)]	Loss: 0.586298

Test set: Average loss: 0.0053, Accuracy: 7919/10000 (79%)

Train Epoch: 280 [0/50000 (0%)]	Loss: 0.641738
Train Epoch: 280 [25600/50000 (51%)]	Loss: 0.603934

Test set: Average loss: 0.0053, Accuracy: 7926/10000 (79%)

Train Epoch: 281 [0/50000 (0%)]	Loss: 0.579712
Train Epoch: 281 [25600/50000 (51%)]	Loss: 0.599054

Test set: Average loss: 0.0053, Accuracy: 7926/10000 (79%)

Train Epoch: 282 [0/50000 (0%)]	Loss: 0.572755
Train Epoch: 282 [25600/50000 (51%)]	Loss: 0.542889

Test set: Average loss: 0.0053, Accuracy: 7930/10000 (79%)

Train Epoch: 283 [0/50000 (0%)]	Loss: 0.601990
Train Epoch: 283 [25600/50000 (51%)]	Loss: 0.553783

Test set: Average loss: 0.0053, Accuracy: 7918/10000 (79%)

Train Epoch: 284 [0/50000 (0%)]	Loss: 0.

In [0]:
# if in Google Colab, download your model with this
from google.colab import files
files.download("best.pt")

In [0]:
a=3
def test():
  print(a)
test()

3
